### Importing Libraries and Database

In [2]:
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#Defining the database and making the type of some variables explicit as categorical
data = pd.read_csv('anime.csv', dtype = {'startYr': 'category', 'finishYr': 'category', }, decimal = '.')


ModuleNotFoundError: No module named 'pandas'

### Data Processing

- title (String): Anime title - Ex: "Attack on Titan 3rd Season: Part II"
- description (String): Anime description - Ex: "Naruto the orphan has always wanted a family, [...]"
- contentWarn (Lista de String): Content warning - Ex: ['Violence']


In [2]:
# Excluding columns that will not be used -> "description" and "contentWarn" (90% of data is "[]")
data = data.drop(['title', 'description', 'contentWarn', 'studios', 'tags'], axis = 1)

- mediaType (String-Categórica): Anime media type - Categorias: [TV, TV Special, Movie, OVA, Music Video, Web, DVD Special]



In [3]:
# Inserting value 'TV'(mode) in column "mediaType" where it has value null
data.mediaType.fillna('TV', inplace = True)

# Changing the values ​​"TV" to 0, "Movie" to 1, "OVA" to 2, "Music Video" to 3, "Web" to 4, "DVD Special" to 5, "Other" to 6, "TV Special " by 7 in the column "mediaType"
data.loc[data.mediaType == 'TV', 'mediaType'] = 0
data.loc[data.mediaType == 'Movie', 'mediaType'] = 1
data.loc[data.mediaType == 'OVA', 'mediaType'] = 2
data.loc[data.mediaType == 'Music Video', 'mediaType'] = 3
data.loc[data.mediaType == 'Web', 'mediaType'] = 4
data.loc[data.mediaType == 'DVD Special', 'mediaType'] = 5
data.loc[data.mediaType == 'Other', 'mediaType'] = 6
data.loc[data.mediaType == 'TV Special', 'mediaType'] = 7

enc = OneHotEncoder(drop = 'first')
enc_df = pd.DataFrame(enc.fit_transform(data[['mediaType']]).toarray())
data = data.join(enc_df)
data.drop(['mediaType'], axis = 1, inplace = True)

data = data.rename(columns = {0: 'feature0', 1: 'feature1', 2: 'feature2', 3: 'feature3', 4: 'feature4', 5: 'feature5', 6: 'feature6'})

- eps (Int): Number of episodes - Interval: 1-2527

- duration (Int): Average anime episode length, given in minutes - Interval: 1-235



In [4]:
# Inserting value '12' (mode) in column "eps" where it has null value
data['eps'] =  data ['eps'].astype('float64')
data['eps'].fillna(12, inplace = True)

# Inserting value '22' (mode) in column "duration" where it has null value
data['duration'] = data['duration'].astype('float64')
data['duration'].fillna(22, inplace = True)

# Transforming the "duration" column into the total time it would take to watch the entire anime
data['duration'] = data['eps'] * data['duration']

- watched (Int): Number of people who attended - Interval: 0-162000


In [5]:
# Inserting value "1471"(fashion) in column "watched" where it has null value
data.watched.fillna(1471, inplace = True)

- rating (Int): end note of the anime - Interval: 0.84-4.7

- votes (Int): Number of votes to contribute to the final grade - Interval: 10-131000




In [6]:
# Inserting the median (=~2.965) in the "rating" column where it has null value
data.rating.fillna(2.9, inplace = True)

# Inserting the median (218) in the "votes" column where it has null value
data.votes.fillna(data.votes.median(), inplace = True)
print(data.votes.median())


218.0


- ongoing (Boolean): Whether or not the anime is in production yet - True(2%) e False(98%)


- startYr (Int): Release year - Interval: 1907-2026


- finishYr (Int): Anime ending year - Interval: 1907-2026



In [7]:
# Changing 'True' to '1' and 'False' to 0 in column "ongoing"
data.loc[data.ongoing == False, 'ongoing'] = 0
data.loc[data.ongoing == True, 'ongoing'] = 1

# Inserting the value '2016'(mode) in the column "startYr" where it has a null value and transforming it into a categorical variable
data.startYr.fillna(data.startYr.mode(), inplace = True)
data.startYr = data.startYr.astype('category').cat.codes

# Inserting the value '2016'(fashion) in the column "finishYr" where it has a null value and transforming it into a categorical variable
data.finishYr.fillna(data.finishYr.mode(), inplace = True)
data.finishYr = data.finishYr.astype('category').cat.codes

- sznOfRelease (String-Categórica): Season of the year it was released - Categorias: [Spring, Winter, Fall, Summer]


In [8]:
# Inserting the value 'Spring' (fashion) in the column "sznOfRelease" where it has null value
data.sznOfRelease.fillna('Spring', inplace = True)

# Changing 'Spring' to '0', 'Winter' to 1, 'Fall' to 2 and 'Summer' to 3 in column "sznOfRelease"
data.loc[data.sznOfRelease == 'Spring', 'sznOfRelease'] = 0
data.loc[data.sznOfRelease == 'Fall', 'sznOfRelease'] = 1
data.loc[data.sznOfRelease == 'Winter', 'sznOfRelease'] = 2
data.loc[data.sznOfRelease == 'Summer', 'sznOfRelease'] = 3

enc = OneHotEncoder(drop = 'first')
enc_df = pd.DataFrame(enc.fit_transform(data[['sznOfRelease']]).toarray())
data = data.join(enc_df)
data.drop(['sznOfRelease'], axis = 1)

data = data.rename(columns = {0: 'feature7', 1: 'feature8', 2: 'feature9', 3: 'feature10'})

Nothing was done with the following variables as they already came with no missing values:
- watching (Int): Number of people watching - Interval: 0-74500

- wantWatch (Int): Number of people who want to watch - Interval: 0-28500

- dropped (Int):  Number of people who started but stopped watching - Interval: 0-19500



### Machine Learning Algorithm

In [9]:
y = data.rating
x = data.drop(['rating'], axis = 1)

Xtrain, Xval, Ytrain, Yval = train_test_split(x, y, test_size = 0.3, random_state = 0)

scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xval_scaled = scaler.transform(Xval)

In [10]:
# ------------------------------------------- Linear Regression -------------------------------------------
modelo = LinearRegression(n_jobs = 4)
modelo.fit(Xtrain_scaled, Ytrain)

y_pred = modelo.predict(Xtrain_scaled)

score = r2_score(Ytrain,y_pred)
p = modelo.predict(Xval_scaled)

print("Acurância: {:.3f}%".format(round(score*100, 2)) ) #35,07%
print("Erro quadrático médio: " + str(round(mean_squared_error(Yval, p), 2))) #MSE 0.37
print("Erro médio absoluto: " + str(round(mean_absolute_error(Yval, p), 2))) #MAE 0.48
print("Erro quadrático médio da raiz: " + str(round(sqrt(mean_squared_error(Yval, p)), 2))) #RMSE 0.61

Acurância: 35.070%
Erro quadrático médio: 0.37
Erro médio absoluto: 0.48
Erro quadrático médio da raiz: 0.61


In [11]:
from sklearn.ensemble import RandomForestRegressor
# ------------------------------------------- Random Forest -------------------------------------------
modelo = RandomForestRegressor(n_estimators = 1000, random_state = 42)
modelo.fit(Xtrain_scaled, Ytrain)

y_pred = modelo.predict(Xtrain_scaled)

score = r2_score(Ytrain,y_pred)
p = modelo.predict(Xval_scaled)

print("Acurância: {:.3f}%".format(round(score*100, 2)) ) #96,19%
print("Erro quadrático médio: " + str(round(mean_squared_error(Yval, p), 2))) #MSE 0.16
print("Erro médio absoluto: " + str(round(mean_absolute_error(Yval, p), 2))) #MAE 0.28
print("Erro quadrático médio da raiz: " + str(round(sqrt(mean_squared_error(Yval, p)), 2))) #RMSE 0.4

Acurância: 96.190%
Erro quadrático médio: 0.16
Erro médio absoluto: 0.28
Erro quadrático médio da raiz: 0.4
